Import

In [2]:
import subprocess
import shlex
import os.path
import sys
import pickle
import datetime
import tweepy
import matplotlib.pyplot as plt 
import numpy as np

Senti Strength Program and Data Location

In [3]:
SentiStrengthLocation = "" #The location of SentiStrength on your computer
SentiStrengthLanguageFolder = "" #The location of the unzipped SentiStrength data files on your computer

Error Check for Senti Strength (from Senti Strength)

In [4]:
if not os.path.isfile(SentiStrengthLocation):
    print("SentiStrength not found at: ", SentiStrengthLocation)
if not os.path.isdir(SentiStrengthLanguageFolder):
    print("SentiStrength data folder not found at: ", SentiStrengthLanguageFolder)

Senti Strength Helper Function (from Senti Strength)

In [5]:
def RateSentiment(sentiString):
    #open a subprocess using shlex to get the command line string into the correct args list format
    p = subprocess.Popen(shlex.split("java -jar '" + SentiStrengthLocation + "' stdin sentidata '" + SentiStrengthLanguageFolder + "'"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    #communicate via stdin the string to be rated. Note that all spaces are replaced with +
    #Can't send string in Python 3, must send bytes
    b = bytes(sentiString.replace(" ","+"), 'utf-8')
    stdout_byte, stderr_text = p.communicate(b)
    #convert from byte
    stdout_text = stdout_byte.decode("utf-8") 
    #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1 -5
    stdout_text = stdout_text.rstrip().replace("\t"," ")
    return stdout_text + " " + sentiString

Other Helper Functions

In [6]:
def get_list_of_users(tweets):

    list_of_user = []

    for tweet in tweets:
        user_id = tweet.user.id
        if user_id not in list_of_user:
            list_of_user.append(user_id)

    return list_of_user

In [57]:
def get_related_tweet(tweets, end_time, time_bound):
   
    time_limit = datetime.timedelta(hours=time_bound)
    lower_bound = end_time - time_limit
    tweet_return = {}
    
    for user in tweets:
        
        friend_tweets = tweets[user]
        
        for tweet in friend_tweets:
        
            tweet_created = tweet.created_at

            if end_time >= tweet_created >= lower_bound:
                tweet_id = tweet.id
                tweet_return[tweet_id] = tweet
            
    return tweet_return
    

Import Tweet and Related Data

In [43]:
#Location of all the ".p" files
directory = "" 
# Where you want to save the coverted files
test_directory = "" 

In [35]:
# Tweets output from the function, get_inital_tweets
initial_tweets = pickle.load(open("", "rb"))

Conversion from Old Format to New Format - ONLY REQUIRED IF YOU USED THE OLD CODE AND HAVE the "temp.p" file

In [41]:
convert = pickle.load(open("L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/temp.p", "rb"))

In [44]:
index = 1
for user in convert:
    print("Converting user " + str(index) + " of " + str(len(convert)))
    index += 1
    
    user_id = user
    file_name = test_directory + str(user_id) + '.p'
    friend_info = convert[user]
    
    pickle.dump(friend_info, open(file_name, "wb"))
        

Converting user 1 of 7
Converting user 1 of 7
Converting user 1 of 7
Converting user 1 of 7
Converting user 1 of 7
Converting user 1 of 7
Converting user 1 of 7


Determine Sentiment of Initial Tweets and Related Tweets

In [ ]:
# TODO - Convert to helper function
sentiment_for_tweet = {}
average_sentiment_for_tweet = {}

data_analyzed = {}
index = 1

# Analyze all the  users in initial_tweets
for t in initial_tweets:
    print('Analyzing user ' + str(index) + " of " + str(len(initial_tweets)))
    index += 1
    
    tweets = initial_tweets[t]
    print("Analyzing " + str(len(tweets['tweets'])) + " tweets")
    
    # Every user have a set of tweets, analyze all of them and return sentiment
    for tweet in tweets['tweets']:
        #print(tweet)
        # Tweet info
        user_id = tweet.user.id
        tweet_id = tweet.id
        time_created = tweet.created_at
        tweet_text = tweet.full_text
        file_name = directory + str(user_id) + '.p'

        # Sentiment for original tweet
        sentiment = RateSentiment(tweet_text)[:4].split()
        sentiment = [int(sentiment[0]), int(sentiment[1])]
        combined = sentiment[0] + sentiment[1]
        sentiment_for_tweet[tweet_id] = {'sentiment': sentiment}
        sentiment_for_tweet[tweet_id]['combined'] = combined
        sentiment_for_tweet[tweet_id]['user'] = user_id

        average_sentiment_for_tweet[tweet_id] = {'sentiment': sentiment}
        average_sentiment_for_tweet[tweet_id]['combined'] = combined
        
        # If we have scapped the data from Twitter, analyze this tweet, otherwise, skip it
        if os.path.exists(file_name):
            if user_id not in data_analyzed:
                data_analyzed[user_id] = []

            data_analyzed[user_id].append(tweet_id)
            
            # Get tweets that are posted at most one hour before this tweet (known as related tweets throughout the code)
            friend_tweets = pickle.load(open(file_name, "rb"))
            related_friend_tweets = get_related_tweet(friend_tweets, time_created, 1)

            if 'friend' not in sentiment_for_tweet[tweet_id]:
                sentiment_for_tweet[tweet_id]['friend'] = {}

            total_sentiment = []
            print("Analyzing " + str(len(related_friend_tweets)) + " related tweets")
            
            # Get the sentiment for all related tweets
            for f in related_friend_tweets:
                f_tweet = related_friend_tweets[f]
                f_tweet_id = f_tweet.id
                f_tweet_text = f_tweet.full_text

                f_sentiment = RateSentiment(f_tweet_text)[:4].split()
                f_sentiment = [int(f_sentiment[0]), int(f_sentiment[1])]
                f_combined = f_sentiment[0] + f_sentiment[1]
                total_sentiment.append(f_combined)

                sentiment_for_tweet[tweet_id]['friend'][f_tweet_id] = {'sentiment': sentiment}
                sentiment_for_tweet[tweet_id]['friend'][f_tweet_id]['combined'] = f_combined


            average_s = sum(total_sentiment)/len(total_sentiment)
            average_sentiment_for_tweet[tweet_id]['friend'] = average_s
            
        # Periodically save the result
        pickle.dump(sentiment_for_tweet, open("sentiment.p", "wb"))
        pickle.dump(average_sentiment_for_tweet, open("avg_sentiment.p", "wb"))
        pickle.dump(data_analyzed, open("data_analyzed.p", "wb"))
    

Analyzing user 1 of 300
Analyzing 60 tweets
Analyzing 75 related tweets


Work In Progress

Extra Code